# Model ResNet

https://www.pluralsight.com/guides/introduction-to-resnet

#### TODOS
1. DONE Debugging, does output make sense?
    1. Resize images
    2. preprocessing fixes
    5. replace scaling by proper function
2. try on leaderboard
3. Include Odometry and fuse into heads
    - Speed
    - Location
4. navigation
5. controller
6. Evaluation on Test set, Modularization



## Dependencies

In [37]:
# MODEL STUFF
import torch
import torch.nn as nn
import torch.optim as optim
#import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

# GENERAL STUFF
import time
import copy
import os
import sys
sys.path.insert(1, 'C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/data_pipeline') # TODO

# DATA ENGINEERING
from data_sampler import WeightedSampler
from dataset import CARLADataset#, CARLADatasetMultiProcessing

## Model

In [38]:
class MyResnet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # ResNet Architecture with pretrained weights, also bigger resnets available
        self.net = torchvision.models.resnet18(weights=True)
        num_ftrs = self.net.fc.in_features

        # Top layer of ResNet which you can modify. We choose Identity to use it as Input for all the heads
        self.net.fc = nn.Identity()
        
        
        # Regression Heads for Throttle, Brake and Steering
        self.thr_head = nn.Sequential(
            nn.Linear(num_ftrs, 1),
            nn.Sigmoid() # [0,1] Range Output
        )
        
        self.brk_head = nn.Sequential(
            nn.Linear(num_ftrs, 1),
            nn.Sigmoid() # [0,1] Range Output
        )
        
        self.str_head = nn.Sequential(
            nn.Linear(num_ftrs, 1),
            nn.Tanh() # [-1,1] Range Output
        )

    # Forward Pass of the Model
    def forward(self, x):
        x = self.net(x)
        #x = self.net.fc(x)
        return self.thr_head(x), self.str_head(x), self.brk_head(x) # 3 Outputs since we have 3 Heads

## Data Loaders, Data Sets

In [39]:
#path_ege_data = os.path.join("..", "..", "data", "Dataset Ege")
train_path = "D:/data/Train"
test_path = "D:/data/Test"

config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake"],
        "seq_len": 1
        }

train_dataset = CARLADataset(root_dir=test_path, config=config, transform=None)
test_dataset = CARLADataset(root_dir=test_path, config=config, transform=None)

weighted_sampler = WeightedSampler(dataset=train_dataset)

batch_size = 64
train_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

### Perprocessing Workarounds

In [40]:
def normalize_batch(tensors):
    liste = []
    for tensor in tensors:
        tensor = preprocess(tensor )#* 1/255
        liste.append(tensor)
    return torch.stack(liste)

In [42]:
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.229, 0.224, 0.225]), # [0.485, 0.456, 0.406]
])

#train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#img = next(iter(train_dataloader))["rgb"].transpose(1,4)[:,:,:,:,0].float()
#img.shape

In [31]:
#normalize_batch(img).shape#[0,:,:,:]

In [32]:
#np.mean(normalize_batch(img).numpy(), axis =(0,2,3))

## Training

In [43]:
# Initialise Model (GPU or CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = MyResnet().cuda() if device else net
net

C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MyResnet(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [44]:
def to_cuda_if_possible(data):
    return data.to(device) if device else data

In [45]:
def forward_pass(data):
    # further preprocessing
    X_rgb = normalize_batch(data["rgb"].transpose(1,4)[:,:,:,:,0]).float() # HOTFIX
    X_throttle = data["throttle"].float()
    X_steer = data["steer"].float()
    X_brake = data["brake"].float()

    # move to GPU
    X_rgb = to_cuda_if_possible(X_rgb)
    X_throttle = to_cuda_if_possible(X_throttle)
    X_steer = to_cuda_if_possible(X_steer)
    X_brake = to_cuda_if_possible(X_brake)

    # compute outputs
    optimizer.zero_grad()

    Y_hat = net(X_rgb)
    Y_hat_throttle = to_cuda_if_possible(Y_hat[0])
    Y_hat_steer = to_cuda_if_possible(Y_hat[1])
    Y_hat_brake = to_cuda_if_possible(Y_hat[2])

    # get labels from data
    Y_throttle = to_cuda_if_possible(data["throttle"].float())
    Y_steer = to_cuda_if_possible(data["steer"].float())
    Y_brake = to_cuda_if_possible(data["brake"].float())

    # Calculate Loss
    loss_throttle = criterion(Y_hat_throttle, Y_throttle)
    loss_steer = criterion(Y_hat_steer, Y_steer)
    loss_brake = criterion(Y_hat_brake, Y_brake)
    loss = sum([loss_throttle, loss_steer, loss_brake])/3
    return loss

In [46]:
# Loss and Optimizer
criterion = nn.L1Loss() # Easy to interpret #nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001) #optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Model Trainer Prototype

In [ ]:
%%time

n_epochs = 3
#print_every = 10
valid_loss_min = np.Inf
val_loss = []
train_loss = []
total_step = len(train_dataloader)

run = True

for epoch in range(1, n_epochs+1):
    
    running_loss = 0.0
    print(f'Epoch {epoch}\n')
    
    # Work through batches
    for batch_idx, data in enumerate(train_dataloader): #data: (['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])

        loss = forward_pass(data)
        
        # Backprop
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (batch_idx) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
        
    # Epoch finished, evaluate network and save if network_learned
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f},')
    batch_loss = 0

    
    # Evaluation on Test set, skipped for now
    with torch.no_grad():
        net.eval()
        
        for batch_idx, data in enumerate(test_dataloader):
            
            loss = forward_pass(data)
            
            batch_loss += loss.item()
        val_loss.append(batch_loss/len(test_dataloader))
        #network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, \n')

        
        if False:#network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')

    # Back to training
    net.train()
    
    

Epoch 1

Epoch [1/3], Step [0/20], Loss: 0.3862
Epoch [1/3], Step [10/20], Loss: 0.2914

train-loss: 0.3069,


### Test predictions

In [ ]:
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
print(next(iter(test_dataloader)).keys())


In [ ]:
idx, X = next(enumerate(test_dataloader))

In [ ]:
data_ = normalize_batch(X["rgb"].transpose(1,4)[:,:,:,:,0]).float().to(device)
target_ = (X["throttle"],X["steer"],X["brake"])

In [ ]:
from torchvision import transforms

img = X["rgb"].transpose(1,4)[0,:,:,:,0]

# Define a transformation to normalize the tensor
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.229, 0.224, 0.225])

# Apply the transformation to the tensor
normalized_img = img#/255#normalize(img)

# Convert the tensor to a PIL image
img_pil = transforms.ToPILImage()(normalized_img)

# Display the image
img_pil.show()


In [ ]:
print(img.shape)

In [ ]:
transforms.ToPILImage(img)
img_pil.show()

In [ ]:
np.shape(data_)

In [ ]:
with torch.no_grad():
    net.eval()
    outputs_ = net(data_)

In [ ]:
i =0

In [ ]:
print(np.round(outputs_[i].cpu().numpy(),1))



In [ ]:
print(np.round(target_[i].cpu().numpy(),1))

In [ ]:
np.mean(abs(outputs_[i].cpu().numpy()-target_[i].cpu().numpy()))

### Vanishing/Exploding Gradients

In [ ]:
"""
for name, param in net.thr_head.named_parameters():
    if param.requires_grad:
        print(name, param.data.cpu().numpy())
"""

In [ ]:
for name, param in net.thr_head.named_parameters():
    if param.requires_grad:
        print(name, np.sum(param.data.cpu().numpy()))

## Saving and Loading

Not suited for leaderboard agents

In [ ]:
#torch.save(net, 'rgb_resnet.pth')

In [ ]:
#net = torch.load('rgb_resnet.pth')

In [ ]:
#net = torch.load('rgb_resnet.pth')

suited for leaderboard agents

In [ ]:
torch.save(net.state_dict(), "rgb_resnet.pth")

In [ ]:
#net = MyResnet()
#net.load_state_dict(torch.load("rgb_resnet.pth"))
#net.cuda()

## Testing Time

In [ ]:
# ohne preprocessing ca 16-17 sekunden. Mit preprocessing ca 37 sekunden
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
at = time.time()
for batch_idx, data in enumerate(train_dataloader):
    data_ = normalize_batch(data["rgb"].transpose(1,4)[:,:,:,:,0]).float()
et = time.time()
print(et-at)
